##  Fetching Data and Data Cleaing

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Basic
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import os
import random
from tqdm.autonotebook import tqdm
import string
import re
from functools import partial
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import functools
from IPython.core.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
#Metadata_df=pd.read_csv('/kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.csv')
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

#Metadata_df = pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv')

In [ ]:
print('shape' ,train_df.shape)

In [ ]:
print('info',train_df.info())

In [ ]:
train_df.head()

# 1.Rendering Data

In [ ]:

def read_append_return(filename, train_files_path=train_files_path, output='text'):
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
tqdm.pandas()
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

In [ ]:
pd.set_option('display.max_colwidth', None)
train_df.text[train_df['Id']=='0007f880-0a9b-492d-9a58-76eb0b0e0bd7']

In [ ]:
for col in train_df.columns:
    print(f"{col}: {len(train_df[col].unique())}")

# 2.Data Visualization

In [ ]:
#textprops={'color':"b"}
import matplotlib.pyplot as plt
train_df['pub_title'].value_counts().iloc[[0,1,2,3,4,5,6,7,8,9]].plot.pie(figsize = (10,10),autopct = '%.2f%%',
                                                                           title = 'Top Ten Pub Title')
plt.title("Top Ten Pub Title", bbox={'facecolor':'0.8', 'pad':5})
plt.savefig("Top Ten Pub Title"+".png", bbox_inches='tight')

In [ ]:

train_df['dataset_title'].value_counts().iloc[[0,1,2,3,4,5,6,7,8,9]].plot.pie(figsize = (10,10), autopct = '%.2f%%',
                                                                           title = 'Top Ten Dataset title')
plt.title("Top Ten Data set title", bbox={'facecolor':'0.8', 'pad':5})
plt.savefig("Top Ten Data set title"+".png", bbox_inches='tight')

In [ ]:

train_df['dataset_label'].value_counts().iloc[[0,1,2,3,4,5,6,7,8,9]].plot.pie(figsize = (10,10), autopct = '%.2f%%',
                                                                           title = 'Top Ten Dataset Label')
plt.title("Top Ten Dataset Label", bbox={'facecolor':'0.8', 'pad':5})
plt.savefig("Top Ten Dataset Label"+".png", bbox_inches='tight')

In [ ]:
duplicateRowsDF = train_df['Id'][train_df.duplicated(['Id'])]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

In [ ]:
dups_ID = train_df.pivot_table(index=['Id'], aggfunc='size')
print (dups_ID)

# 3.Text Preprocessing

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from collections import Counter 
import time
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from gensim import corpora,models,similarities
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.summarization import summarize
from gensim.summarization import keywords
porter = PorterStemmer()

In [ ]:
# getting the lower case , removing stopwords and gettting the word origin
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def text_preprocessing(text):
    text= re.sub('[^a-zA-z0-9\s]','',text)
    text=lower_case(text)
    text=remove_stopwords(text)
    text=stemSentence(text)    
    return text

In [ ]:
'''Combined_data_processing=train_df.copy()
Combined_data_processing['pub_title']=Combined_data_processing['pub_title'].apply(lambda x: text_preprocessing(x))
Combined_data_processing['dataset_title']=Combined_data_processing['dataset_title'].apply(lambda x: text_preprocessing(x))
Combined_data_processing['dataset_label']=Combined_data_processing['dataset_label'].apply(lambda x: text_preprocessing(x))
Combined_data_processing['text']=Combined_data_processing['text'].apply(lambda x: text_preprocessing(x))'''

In [ ]:
#Combined_data_processing.to_csv('processed_train.csv')

In [ ]:
Combined_data_processing=pd.read_csv('../input/coleridge-cosine-similarity-gensim/processed_train.csv')
Combined_data_processing.head()

In [ ]:
'''KeyWords=[]
for i in range(0,len(Combined_data_processing)):
    KeyWords.append(remove_stopwords(str(keywords(Combined_data_processing['text'][i]))))'''

In [ ]:
'''corpus_text = [] # intializing list of collection of  clean reviews

for i in range(0, 33384):
    for word in str(KeyWords['KeyWords'][i]).split():
        corpus_text.append(word)'''

In [ ]:

#len(KeyWords)

In [ ]:
 #removing the words less than four chars 
#corpus_text_modified=[i for i in corpus_text if 5 <=  len(i)]

In [ ]:
'''unique_string=(" ").join(KeyWords)
wordcloud = WordCloud(width = 1000, height = 500 ,max_words=10000).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("your_file_name"+".png", bbox_inches='tight')
plt.show()
plt.close()'''

# 4.Word Clouds

In [ ]:
corpus_pub_title = [] # intializing list of collection of  clean reviews

for i in range(0, len(Combined_data_processing)):
    for word in str(Combined_data_processing['pub_title'][i]).split():
        corpus_pub_title.append(word)

In [ ]:
''''removed_words= ['nan', 'missing','miss']  
for word in list(corpus_pub_title):  # iterating on a copy since removing will mess things up
    if word in removed_words:
        corpus_pub_title.remove(word)

In [ ]:
corpus_pub_title=[i for i in corpus_pub_title if 3 <=  len(i)]

In [ ]:
unique_string=(" ").join(corpus_pub_title)
wordcloud = WordCloud(background_color='white',width = 1000, height = 500 ,max_words=10000).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("Authors_Words"+".png", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
corpus_dataset_title = [] # intializing list of collection of  clean reviews

for i in range(0, len(Combined_data_processing)):
    for word in str(Combined_data_processing['dataset_title'][i]).split():
        corpus_dataset_title.append(word)

In [ ]:
unique_string=(" ").join(corpus_dataset_title)
wordcloud = WordCloud(background_color='white',width = 1000, height = 500 ,max_words=10000).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("Titles_Words"+".png", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
corpus_dataset_label = [] # intializing list of collection of  clean reviews

for i in range(0, len(Combined_data_processing)):
    for word in str(Combined_data_processing['dataset_label'][i]).split():
        corpus_dataset_label.append(word)

In [ ]:
unique_string=(" ").join(corpus_dataset_label)
wordcloud = WordCloud(background_color='white',width = 1000, height = 500 ,max_words=30000).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("abstract_words"+".png", bbox_inches='tight')
plt.show()
plt.close()

# 5.  Most Frequent Words

In [ ]:
title_font = {'fontname':'Arial', 'size':'18', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'14'}
#corpus_title=[i for i in corpus_title if 4 <=  len(i)]
counter_title = Counter(corpus_dataset_title) 
  
# most_common() produces k frequently encountered 
# input values and their respective counts. 
most_occur = dict(counter_title.most_common(100)) 
  
print(most_occur) 

labels, values = zip(*most_occur.items())

# sort your values in descending order
indSort = np.argsort(values)[::-1]

# rearrange your data
labels = np.array(labels)[indSort]
values = np.array(values)[indSort]

indexes = np.arange(len(labels))
plt.figure(figsize=(20,40))
plt.barh(labels, values)
plt.title('The most frequent words in Articles Titles',**title_font,bbox={'facecolor':'0.8', 'pad':5})
plt.xlabel('Number of Words',**axis_font,color='black')
plt.ylabel('Top 100 Words in Titles',**axis_font,color='black')
plt.xticks(color='black')
plt.yticks(color='black')
plt.savefig("title most frequent Words"+".png", bbox_inches='tight')
# add labels
#plt.xticks(indexes + bar_width, labels)
plt.show()

In [ ]:
counter_abstract = Counter(corpus_dataset_label) 
  
# most_common() produces k frequently encountered 
# input values and their respective counts. 
most_occur = dict(counter_abstract.most_common(100)) 
  
print(most_occur) 

labels, values = zip(*most_occur.items())

# sort your values in descending order
indSort = np.argsort(values)[::-1]

# rearrange your data
labels = np.array(labels)[indSort]
values = np.array(values)[indSort]

indexes = np.arange(len(labels))
plt.figure(figsize=(20,40))
plt.barh(labels, values)
plt.title('The most frequent words in Articles Abstract',**title_font,bbox={'facecolor':'0.8', 'pad':5})
plt.xlabel('Number of Words',**axis_font,color='black')
plt.ylabel('Top 100 Words in Abstracts',**axis_font,color='black')
plt.xticks(color='black')
plt.yticks(color='black')
plt.savefig("abstract most frequent Words"+".png", bbox_inches='tight')
# add labels
#plt.xticks(indexes + bar_width, labels)
plt.show()

In [ ]:
title_font = {'fontname':'Arial', 'size':'18', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'14'}
#corpus_authors=[i for i in corpus_authors if 2 <=  len(i)]
corpus_authors = Counter(corpus_pub_title) 
  
# most_common() produces k frequently encountered 
# input values and their respective counts. 
most_occur = dict(corpus_authors.most_common(100)) 
  
print(most_occur) 

labels, values = zip(*most_occur.items())

# sort your values in descending order
indSort = np.argsort(values)[::-1]

# rearrange your data
labels = np.array(labels)[indSort]
values = np.array(values)[indSort]

indexes = np.arange(len(labels))
plt.figure(figsize=(20,40))
plt.barh(labels, values)
plt.title('The most frequent words in Articles Abstract',**title_font,bbox={'facecolor':'0.8', 'pad':5})
plt.xlabel('Number of Words',**axis_font,color='black')
plt.ylabel('Top 100 Words in Abstracts',**axis_font,color='black')
plt.xticks(color='black')
plt.yticks(color='black')
plt.savefig("abstract most frequent Words"+".png", bbox_inches='tight')
# add labels
#plt.xticks(indexes + bar_width, labels)
plt.show()

# 6. Modelling

In [ ]:
#Implementing TFIDF 
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(Combined_data_processing['text'].values)

#implementing K-Means
k = 9
kmeans = MiniBatchKMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(X.toarray())
            
print(pca.components_)
print(pca.explained_variance_)

In [ ]:
#implementing K-Means Clustering
title_font = {'fontname':'Arial', 'size':'18', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'14'}
import seaborn as sns
sns.set(rc={'figure.figsize':(15,15)})

# colors
palette = sns.color_palette("muted", len(set(y_pred)))

# plot
sns.scatterplot(pca_result[:,0], pca_result[:,1], hue=y_pred,sizes =100, legend='full',palette=palette)
plt.title("PCA - Clustered (K-Means)",**title_font,bbox={'facecolor':'0.8', 'pad':5})
plt.savefig("Kmeans"+".png", bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=pca_result[:,0], 
    ys=pca_result[:,1], 
    zs=pca_result[:,2], 
    c=y_pred, 
    cmap='tab10'
)
ax.set_xlabel('pca-1')
ax.set_ylabel('pca-2')
ax.set_zlabel('pca-3')

plt.title("PCA - Clustered (K-Means)-3D",**title_font,bbox={'facecolor':'0.9', 'pad':2})
plt.savefig("Kmeans_3D"+".png", bbox_inches="tight")
plt.show()

# 7. Model Evaluation

In [ ]:
#Getting the best number of clusters using elbow method
import time
def elbow_plot(data, start_K, end_K, step):
    '''
    Generate an elbow plot to find optimal number of clusters
    graphing K values from start_K to end_K every step value
    
    INPUT: 
        data: Demographics DataFrame
        start_K: Inclusive starting value for cluster number
        end_K: Exclusive stopping value for cluster number
        step: Step value between start_K and end_K
    OUTPUT: Trimmed and cleaned demographics DataFrame
    '''
    score_list = []

    for i in range(start_K, end_K, step):
        print(i)
        start = time.time()
        kmeans = MiniBatchKMeans(i)
        model = kmeans.fit(data)
        score = model.score(data)
        score_list.append(abs(score))
        end = time.time()
        elapsed_time = end - start
        print(elapsed_time)

    plt.plot(range(start_K, end_K, step), 
    score_list, linestyle='--', marker='o', color='b');
    plt.xlabel('# of clusters K');
    plt.ylabel('Sum of squared errors');
    plt.title('SSE vs. K');
    plt.savefig('elbow_plot.png')
elbow_plot(pca_result, 1, 20, 1)

## Conclusion
1. -This scatter plot is generated from Articles text , each article text is a feature. 
1. -usinig features vector TfidfVectorizer. 
1. -Dimensionality Reduction using PCA.
1. -generating clustering using k-Means where k=9 (the best value as elbow plot).
1. -Topic Modeling is done on each cluster to get the keywords per cluster.

# to be continued......